In [1]:
import sys
sys.path.append('../../../src')
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'

In [2]:
from mindspore import context, nn
from sponge import ForceField, set_global_units, WithEnergyCell, UpdaterMD
from sponge.system import get_molecule
from sponge.control import SETTLE, LeapFrog, Langevin
from sponge.core import Sponge
from sponge.callback import WriteH5MD, RunInfo
from sponge.function import VelocityGenerator

In [3]:
set_global_units('nm', 'kj/mol')
# context.set_context(mode=context.GRAPH_MODE, device_target='Ascend', device_id=0)
context.set_context(mode=context.GRAPH_MODE, device_id=0)

## 从文件中导入的分子

MindSponge支持了pdb文件和mol2格式文件的分子输入，如果是纯蛋白体系，可以使用MindSponge中自带的Protein来加载一个pdb文件。如果是混合体系的pdb文件，可以通过一个get_molecule的函数来进行加载，得到的也是一个Molecule的基础类型。在mol对象可以调用内部函数来进行填充水分子的操作，可以指定盒子，或者指定边缘宽度，都是支持的。

In [5]:
pdb_name = './case1.pdb'
out_pdb = './case1_add_water.pdb'

mol = get_molecule(pdb_name, template=['protein0.yaml'], rebuild_hydrogen=True)
mol.fill_water(edge=0.4, template='water.tip3p.yaml', pdb_out=out_pdb)

[MindSPONGE] Adding 57 hydrogen atoms for the protein molecule in 0.01 seconds.
[MindSPONGE] The box size used when filling water is: [2.1184928 2.285693  1.549593 ]
[MindSPONGE] The edge gap along x axis is 0.4.
[MindSPONGE] The edge gap along y axis is 0.3999999523162842.
[MindSPONGE] The edge gap along z axis is 0.4.
[MindSPONGE] Totally 172 waters is added to the system!
[MindSPONGE] Adding water molecule task finished!


_MoleculeFromPDB<>

In [6]:
print('The total number of atoms is: ', mol.num_atoms)

The total number of atoms is:  573


In [7]:
print('The residue types in mol are: ', mol.residue_name)

The residue types in mol are:  ['NALA' 'ARG' 'ALA' 'CALA' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT' 'WAT'
 'WAT' 'WAT' 'WAT'

## 多力场加载

MindSponge支持在ForceField中配置多个力场输入，可以通过配置parameters参数来实现。

In [8]:
energy = ForceField(mol, parameters=['AMBER.FF99SB', 'TIP3P'], use_pme=True)

min_opt = nn.Adam(mol.trainable_params(), 1e-03)
sim = WithEnergyCell(mol, energy)
md = Sponge(sim, optimizer=min_opt)

run_info = RunInfo(200)
md.run(2000, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-08-15 10:32:29
[MindSPONGE] Step: 0, E_pot: 25184.45
[MindSPONGE] Step: 200, E_pot: 8776.982
[MindSPONGE] Step: 400, E_pot: 7675.1484
[MindSPONGE] Step: 600, E_pot: 7315.301
[MindSPONGE] Step: 800, E_pot: 7118.0225
[MindSPONGE] Step: 1000, E_pot: 7008.8877
[MindSPONGE] Step: 1200, E_pot: 6904.9062
[MindSPONGE] Step: 1400, E_pot: 6850.5913
[MindSPONGE] Step: 1600, E_pot: 6806.5317
[MindSPONGE] Step: 1800, E_pot: 6749.9697
[MindSPONGE] Finished simulation at 2023-08-15 10:33:16
[MindSPONGE] Simulation time: 47.66 seconds.
--------------------------------------------------------------------------------


## UpdaterMD参数配置

除了通过便捷方式，输入一些字符串给UpdaterMD识别之外（例如配置thermostat='langevin'，constraint='all-bonds'之类），我们还可以直接输入一些Controller对象给UpdaterMD，这样就方便自己去定义一些算法。

In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(mol.shape, mol.atom_mass)

updater = UpdaterMD(
    mol,
    time_step=2e-3,
    velocity=velocity,
    integrator=LeapFrog(mol),
    thermostat=Langevin(mol, temperature=300),
    constraint=SETTLE(mol)
)

md.change_optimizer(updater)

cb_h5md = WriteH5MD(mol, 'tutorial_c04.h5md', save_freq=300, save_last_pdb='tutorial_c04.pdb', write_image=False)

run_info = RunInfo(300)
md.run(3000, callbacks=[run_info, cb_h5md])

[MindSPONGE] The settle constraint is used for the molecule system.
[MindSPONGE] Started simulation at 2023-08-15 10:33:28
[MindSPONGE] Step: 0, E_pot: 6707.88, E_kin: 2081.7134, E_tot: 8789.594, Temperature: 291.8095, Pressure: -5752.4316, Volume: 7.5034766
[MindSPONGE] Step: 300, E_pot: 7731.6914, E_kin: 1266.357, E_tot: 8998.049, Temperature: 253.84624, Pressure: -21191.453, Volume: 7.5034766
[MindSPONGE] Step: 600, E_pot: 7722.826, E_kin: 1284.9515, E_tot: 9007.777, Temperature: 257.57358, Pressure: -19972.49, Volume: 7.5034766
[MindSPONGE] Step: 900, E_pot: 7712.3105, E_kin: 1314.9517, E_tot: 9027.262, Temperature: 263.58722, Pressure: -19159.764, Volume: 7.5034766
[MindSPONGE] Step: 1200, E_pot: 7763.4526, E_kin: 1242.8066, E_tot: 9006.26, Temperature: 249.12547, Pressure: -20794.451, Volume: 7.5034766
[MindSPONGE] Step: 1500, E_pot: 7718.5366, E_kin: 1277.3347, E_tot: 8995.871, Temperature: 256.04678, Pressure: -21711.129, Volume: 7.5034766
[MindSPONGE] Step: 1800, E_pot: 7681.7